In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")

df_ft = pd.read_csv('features_n_target.csv')
df_ft.columns.to_list()

['customer_id',
 'monetary_DNN',
 'monetary_btyd',
 'frequency_DNN',
 'frequency_btyd',
 'frequency',
 't',
 'time_between',
 'avg_basket_value',
 'avg_basket_size',
 'cnt_returns',
 'has_returned',
 'frequency_btyd_clipped',
 'monetary_btyd_clipped',
 'target_monetary_clipped',
 'target_monetary']

In [2]:
features = [
 'monetary_DNN',
 'frequency_DNN',
 'frequency',
 't',
 'time_between',
 'avg_basket_value',
 'avg_basket_size',
 'cnt_returns',
 'has_returned']
numeric_f = ['monetary_DNN','frequency_DNN','frequency','t',
              'time_between','avg_basket_value','avg_basket_size','cnt_returns']
cate_f = ['has_returned']

target = ['target_monetary']

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train_full,X_test,y_train_full,y_test = train_test_split(df_ft[features],df_ft[target],test_size = 0.2,random_state = 42)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,test_size = 0.15, random_state = 42)

scaler = StandardScaler()
# X_train_full = scaler.fit_transform(X_train_full)
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

train_num,train_cate = X_train[:,:-1],X_train[:,-1]
valid_num,valid_cate = X_valid[:,:-1],X_valid[:,-1]
test_num,test_cate = X_test[:,:-1],X_test[:,-1]

# Fine tuning 

### Talos  

In [49]:
def widendeep_model(x_train,y_train,x_valid,y_valid,params):

    input_num = tf.keras.layers.Input(shape = (8,),name = 'deep_input')
    input_cate= tf.keras.layers.Input(shape = (1,),name = 'wide_input')


    hidden_= tf.keras.layers.Dense(params['units'],
                           kernel_initializer = params['initial'], 
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_num)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    
    
    hidden_ = tf.keras.layers.Dense(params['units'],
                       kernel_initializer = params['initial'], 
                       kernel_regularizer=tf.keras.regularizers.l2(0.01))(hidden_)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    hidden_ = tf.keras.layers.Dropout(params['dropout'])(hidden_)


    concat = tf.keras.layers.concatenate([input_cate,hidden_])
    output = tf.keras.layers.Dense(1,name = 'output')(concat)

    model = tf.keras.Model(inputs = [input_cate,input_num],outputs=[output])
    optimizer = tf.keras.optimizers.Adam(learning_rate = params['learning_rate'],beta_1=0.9, beta_2=0.999)
    metrics = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer = optimizer,loss='mse',metrics=metrics)

    out = model.fit(x= x_train,
                    y= y_train,
                    validation_data=[x_valid,y_valid],
                    epochs=150,
                    batch_size=params['batch_size'],
                    verbose=0)
    
    return out,model

In [59]:
import talos
# from keras.utils import plot_model
from scipy.stats import reciprocal
input_shape=[8,1]
learning_rate=reciprocal(3e-4,3e-2).rvs(100).tolist()
units = [128,64,32,16]
initial =['lecun_normal','glorot_uniform','he_normal']
batch_size=[12,16,20]
dropout = [0.1,0.15,0.2,0.25]

params = {'units':units,
         'initial':initial,
         'learning_rate':learning_rate,
         'batch_size':batch_size,
         'dropout':dropout} 


scan_object = talos.Scan(x=[X_train[:,8].reshape(-1,1),X_train[:,:8]],
                         y=y_train,
                         x_val=[X_valid[:,8].reshape(-1,1),X_valid[:,:8]],
                         y_val=y_valid,
                         params=params,
                         model=widendeep_model,experiment_name='try')












  0%|          | 0/14400 [00:00<?, ?it/s]

ValueError: in user code:

    /Users/hengzizhao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /Users/hengzizhao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/hengzizhao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/hengzizhao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/hengzizhao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/hengzizhao/o

In [109]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

units_list = [[128,64],[64,64],[64,32],[32,32],[32,16],[16,16]]
learning_rates=reciprocal(3e-4,3e-2).rvs(100).tolist()
dropout =0.15
batch_size = 12
params_history = []
for units in units_list:
    for learning_rate in learning_rates:
        model = widendeep_model(units,learning_rate)
        history = model.fit((train_cate.reshape(-1,1),train_num), 
                            y_train, 
                            epochs=200, 
                            shuffle=True,
                            batch_size = batch_size,
                            validation_data = ((valid_cate.reshape(-1,1),valid_num),y_valid),
                            callbacks=[earlystop]
         )
        params_history.append([units,learning_rate,history.history['val_root_mean_squared_error']])

Epoch 1/200
95/95 [==============================] - 1s 2ms/step - loss: 157362560.0000 - root_mean_squared_error: 12544.4238 - val_loss: 123635360.0000 - val_root_mean_squared_error: 11119.1436
Epoch 2/200
95/95 [==============================] - 0s 1ms/step - loss: 157271088.0000 - root_mean_squared_error: 12540.7773 - val_loss: 123440544.0000 - val_root_mean_squared_error: 11110.3799
Epoch 3/200
95/95 [==============================] - 0s 1ms/step - loss: 157165104.0000 - root_mean_squared_error: 12536.5518 - val_loss: 123219568.0000 - val_root_mean_squared_error: 11100.4307
Epoch 4/200
95/95 [==============================] - 0s 1ms/step - loss: 157057376.0000 - root_mean_squared_error: 12532.2529 - val_loss: 122976064.0000 - val_root_mean_squared_error: 11089.4570
Epoch 5/200
95/95 [==============================] - 0s 1ms/step - loss: 156900192.0000 - root_mean_squared_error: 12525.9805 - val_loss: 122673152.0000 - val_root_mean_squared_error: 11075.7910
Epoch 6/200
95/95 [======

Epoch 43/200
95/95 [==============================] - 0s 1ms/step - loss: 136254288.0000 - root_mean_squared_error: 11672.8008 - val_loss: 87294784.0000 - val_root_mean_squared_error: 9343.1670
Epoch 44/200
95/95 [==============================] - 0s 1ms/step - loss: 135360080.0000 - root_mean_squared_error: 11634.4336 - val_loss: 86164896.0000 - val_root_mean_squared_error: 9282.5049
Epoch 45/200
95/95 [==============================] - 0s 1ms/step - loss: 135634768.0000 - root_mean_squared_error: 11646.2344 - val_loss: 84400408.0000 - val_root_mean_squared_error: 9186.9688
Epoch 46/200
95/95 [==============================] - 0s 1ms/step - loss: 134705440.0000 - root_mean_squared_error: 11606.2676 - val_loss: 83728920.0000 - val_root_mean_squared_error: 9150.3506
Epoch 47/200
95/95 [==============================] - 0s 1ms/step - loss: 133342968.0000 - root_mean_squared_error: 11547.4229 - val_loss: 80968984.0000 - val_root_mean_squared_error: 8998.2764
Epoch 48/200
95/95 [==========

95/95 [==============================] - 0s 1ms/step - loss: 108361136.0000 - root_mean_squared_error: 10409.6641 - val_loss: 35793128.0000 - val_root_mean_squared_error: 5982.7354
Epoch 128/200
95/95 [==============================] - 0s 1ms/step - loss: 108743216.0000 - root_mean_squared_error: 10428.0020 - val_loss: 34906528.0000 - val_root_mean_squared_error: 5908.1738
Epoch 129/200
95/95 [==============================] - 0s 1ms/step - loss: 106586008.0000 - root_mean_squared_error: 10324.0498 - val_loss: 40537460.0000 - val_root_mean_squared_error: 6366.9033
Epoch 130/200
95/95 [==============================] - 0s 1ms/step - loss: 110424928.0000 - root_mean_squared_error: 10508.3262 - val_loss: 37766452.0000 - val_root_mean_squared_error: 6145.4414
Epoch 131/200
95/95 [==============================] - 0s 2ms/step - loss: 106619400.0000 - root_mean_squared_error: 10325.6670 - val_loss: 38122260.0000 - val_root_mean_squared_error: 6174.3218
Epoch 132/200
95/95 [==================

95/95 [==============================] - 0s 1ms/step - loss: 105026104.0000 - root_mean_squared_error: 10248.2236 - val_loss: 30854650.0000 - val_root_mean_squared_error: 5554.6919
Epoch 12/200
95/95 [==============================] - 0s 1ms/step - loss: 100474768.0000 - root_mean_squared_error: 10023.7070 - val_loss: 29771400.0000 - val_root_mean_squared_error: 5456.3130
Epoch 13/200
95/95 [==============================] - 0s 1ms/step - loss: 101708960.0000 - root_mean_squared_error: 10085.0830 - val_loss: 16937160.0000 - val_root_mean_squared_error: 4115.4722
Epoch 14/200
95/95 [==============================] - 0s 1ms/step - loss: 102127968.0000 - root_mean_squared_error: 10105.8359 - val_loss: 22917918.0000 - val_root_mean_squared_error: 4787.2603
Epoch 15/200
95/95 [==============================] - 0s 1ms/step - loss: 93031656.0000 - root_mean_squared_error: 9645.2900 - val_loss: 16183557.0000 - val_root_mean_squared_error: 4022.8716
Epoch 16/200
95/95 [=========================

Epoch 54/200
95/95 [==============================] - 0s 1ms/step - loss: 48372176.0000 - root_mean_squared_error: 6954.9946 - val_loss: 18611528.0000 - val_root_mean_squared_error: 4314.0830
Epoch 55/200
95/95 [==============================] - 0s 2ms/step - loss: 45355132.0000 - root_mean_squared_error: 6734.6069 - val_loss: 18098138.0000 - val_root_mean_squared_error: 4254.1650
Epoch 56/200
95/95 [==============================] - 0s 2ms/step - loss: 42331220.0000 - root_mean_squared_error: 6506.2271 - val_loss: 15967796.0000 - val_root_mean_squared_error: 3995.9448
Epoch 57/200
95/95 [==============================] - 0s 1ms/step - loss: 39631560.0000 - root_mean_squared_error: 6295.3428 - val_loss: 15160320.0000 - val_root_mean_squared_error: 3893.5962
Epoch 58/200
95/95 [==============================] - 0s 1ms/step - loss: 63908476.0000 - root_mean_squared_error: 7994.2642 - val_loss: 14626957.0000 - val_root_mean_squared_error: 3824.4900
Epoch 59/200
95/95 [====================

...

Epoch 192/200
95/95 [==============================] - 0s 1ms/step - loss: 76704696.0000 - root_mean_squared_error: 8758.1201 - val_loss: 3492123.7500 - val_root_mean_squared_error: 1868.7166
Epoch 193/200
95/95 [==============================] - 0s 1ms/step - loss: 80168032.0000 - root_mean_squared_error: 8953.6592 - val_loss: 3230108.7500 - val_root_mean_squared_error: 1797.2443
Epoch 194/200
95/95 [==============================] - 0s 1ms/step - loss: 76359024.0000 - root_mean_squared_error: 8738.3643 - val_loss: 3653000.0000 - val_root_mean_squared_error: 1911.2765
Epoch 195/200
95/95 [==============================] - 0s 1ms/step - loss: 77299336.0000 - root_mean_squared_error: 8792.0039 - val_loss: 3768928.0000 - val_root_mean_squared_error: 1941.3668
Epoch 196/200
95/95 [==============================] - 0s 1ms/step - loss: 77474864.0000 - root_mean_squared_error: 8801.9785 - val_loss: 3643935.0000 - val_root_mean_squared_error: 1908.9033
Epoch 197/200
95/95 [===================

95/95 [==============================] - 0s 1ms/step - loss: 118261704.0000 - root_mean_squared_error: 10874.8193 - val_loss: 45310540.0000 - val_root_mean_squared_error: 6731.3105
Epoch 77/200
95/95 [==============================] - 0s 1ms/step - loss: 115085952.0000 - root_mean_squared_error: 10727.8125 - val_loss: 48305220.0000 - val_root_mean_squared_error: 6950.1953
Epoch 78/200
95/95 [==============================] - 0s 1ms/step - loss: 116349344.0000 - root_mean_squared_error: 10786.5352 - val_loss: 46661424.0000 - val_root_mean_squared_error: 6830.9170
Epoch 79/200
95/95 [==============================] - 0s 1ms/step - loss: 111632288.0000 - root_mean_squared_error: 10565.6172 - val_loss: 42602256.0000 - val_root_mean_squared_error: 6527.0400
Epoch 80/200
95/95 [==============================] - 0s 1ms/step - loss: 113567624.0000 - root_mean_squared_error: 10656.8115 - val_loss: 50081824.0000 - val_root_mean_squared_error: 7076.8501
Epoch 81/200
95/95 [=======================

95/95 [==============================] - 0s 1ms/step - loss: 94562232.0000 - root_mean_squared_error: 9724.3115 - val_loss: 20882776.0000 - val_root_mean_squared_error: 4569.7666
Epoch 161/200
95/95 [==============================] - 0s 1ms/step - loss: 94798912.0000 - root_mean_squared_error: 9736.4727 - val_loss: 17833604.0000 - val_root_mean_squared_error: 4222.9849
Epoch 162/200
95/95 [==============================] - 0s 1ms/step - loss: 93755960.0000 - root_mean_squared_error: 9682.7666 - val_loss: 14076381.0000 - val_root_mean_squared_error: 3751.8491
Epoch 163/200
95/95 [==============================] - 0s 1ms/step - loss: 95627312.0000 - root_mean_squared_error: 9778.9219 - val_loss: 15257015.0000 - val_root_mean_squared_error: 3906.0217
Epoch 164/200
95/95 [==============================] - 0s 1ms/step - loss: 93784312.0000 - root_mean_squared_error: 9684.2295 - val_loss: 21608024.0000 - val_root_mean_squared_error: 4648.4424
Epoch 165/200
95/95 [============================

Epoch 44/200
95/95 [==============================] - 0s 1ms/step - loss: 156005792.0000 - root_mean_squared_error: 12490.2275 - val_loss: 121261928.0000 - val_root_mean_squared_error: 11011.8994
Epoch 45/200
95/95 [==============================] - 0s 1ms/step - loss: 156018512.0000 - root_mean_squared_error: 12490.7363 - val_loss: 121191800.0000 - val_root_mean_squared_error: 11008.7148
Epoch 46/200
 1/95 [..............................] - ETA: 0s - loss: 12791827.0000 - root_mean_squared_error: 3576.5662

KeyboardInterrupt: 

# Get optimized parameters

In [122]:
df = pd.DataFrame(params_history,columns=['units','learning_rate','loss'])
min_loss = np.zeros(len(df))
for i in range(len(df)):
    loss = min(df.loss[i])
    min_loss[i] = loss
inx = np.argmin(min_loss)
print('optimal units:',df.iloc[inx,0],
     '\noptimal learning rate:',df.iloc[inx,1])

optimal units: [64, 64] 
optimal learning rate: 0.00232087319077393


# Train the multiple imput model

In [4]:
# import keras_tuner as kt
# from keras_tuner import RandomSearch
from scipy.stats import reciprocal


def widendeep_model(units,learning_rate):

    input_num = tf.keras.layers.Input(shape = [8],name = 'deep_input')
    input_cate= tf.keras.layers.Input(shape = [1],name = 'wide_input')


    hidden_= tf.keras.layers.Dense(units[0],
                           kernel_initializer = 'lecun_normal', 
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_num)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    
    
    hidden_ = tf.keras.layers.Dense(units[1],
                       kernel_initializer ='lecun_normal', 
                       kernel_regularizer=tf.keras.regularizers.l2(0.01))(hidden_)
    hidden_ = tf.keras.layers.BatchNormalization()(hidden_)
    hidden_ = tf.keras.activations.relu(hidden_)
    hidden_ = tf.keras.layers.Dropout(dropout)(hidden_)


    concat = tf.keras.layers.concatenate([input_cate,hidden_])
    output = tf.keras.layers.Dense(1,name = 'output')(concat)

    model = tf.keras.Model(inputs = [input_cate,input_num],outputs=[output])
    optimizer = tf.keras.optimizers.Adam(learning_rate,beta_1=0.9, beta_2=0.999)
    metrics = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer = optimizer,loss='mse',metrics=metrics)
    
    return model

In [5]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# tensorboard = keras.callbacks.TensorBoard(run_logdir)


earlystop = keras.callbacks.EarlyStopping(patience=100,
                                          restore_best_weights=True)

units = [64, 64]
learning_rate = 0.00232087319077393
dropout =0.15
batch_size = 12

model = widendeep_model(units,learning_rate)
model.fit((train_cate.reshape(-1,1),train_num), y_train, epochs=300, shuffle=True,
          batch_size = batch_size,
                    validation_data = ((valid_cate.reshape(-1,1),valid_num),y_valid),
                     callbacks=[earlystop]
         )
mse_test = model.evaluate((test_cate.reshape(-1,1),test_num), y_test) 



Epoch 1/300
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-07-23 17:40:34.674059: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-07-23 17:40:34.674216: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


63/95 [==================>...........] - ETA: 0s - loss: 137138778.2698 - root_mean_squared_error: 10685.3850WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x16b9ae4c0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
95/95 [==============================] - 1s 4ms/step - loss: 151996363.4062 - root_mean_squared_error: 11624.13

95/95 [==============================] - 0s 868us/step - loss: 153232904.3503 - root_mean_squared_error: 11702.1912 - val_loss: 33748164.0000 - val_root_mean_squared_error: 5809.3164
Epoch 80/300
95/95 [==============================] - 0s 865us/step - loss: 101223456.8854 - root_mean_squared_error: 9771.7893 - val_loss: 41465460.0000 - val_root_mean_squared_error: 6439.3672
Epoch 81/300
95/95 [==============================] - 0s 857us/step - loss: 85057409.1719 - root_mean_squared_error: 8341.8252 - val_loss: 39547020.0000 - val_root_mean_squared_error: 6288.6421
Epoch 82/300
95/95 [==============================] - 0s 871us/step - loss: 92016881.6589 - root_mean_squared_error: 8476.4637 - val_loss: 36641200.0000 - val_root_mean_squared_error: 6053.1973
Epoch 83/300
95/95 [==============================] - 0s 870us/step - loss: 60747769.2031 - root_mean_squared_error: 7218.4571 - val_loss: 38658848.0000 - val_root_mean_squared_error: 6217.6235
Epoch 84/300
95/95 [====================

Epoch 121/300
95/95 [==============================] - 0s 867us/step - loss: 76874516.1016 - root_mean_squared_error: 8549.0436 - val_loss: 22953254.0000 - val_root_mean_squared_error: 4790.9541
Epoch 122/300
95/95 [==============================] - 0s 875us/step - loss: 111635976.0781 - root_mean_squared_error: 10314.2487 - val_loss: 20734548.0000 - val_root_mean_squared_error: 4553.5195
Epoch 123/300
95/95 [==============================] - 0s 879us/step - loss: 219909113.8542 - root_mean_squared_error: 13462.7170 - val_loss: 23478680.0000 - val_root_mean_squared_error: 4845.4800
Epoch 124/300
95/95 [==============================] - 0s 866us/step - loss: 123375519.2604 - root_mean_squared_error: 10444.2412 - val_loss: 20438642.0000 - val_root_mean_squared_error: 4520.9106
Epoch 125/300
95/95 [==============================] - 0s 867us/step - loss: 75632337.1432 - root_mean_squared_error: 7848.8056 - val_loss: 26003372.0000 - val_root_mean_squared_error: 5099.3496
Epoch 126/300
95/95

Epoch 163/300
95/95 [==============================] - 0s 867us/step - loss: 61862750.9219 - root_mean_squared_error: 6994.5123 - val_loss: 9615988.0000 - val_root_mean_squared_error: 3100.9641
Epoch 164/300
95/95 [==============================] - 0s 893us/step - loss: 93108057.9531 - root_mean_squared_error: 9016.3830 - val_loss: 15737372.0000 - val_root_mean_squared_error: 3967.0339
Epoch 165/300
95/95 [==============================] - 0s 870us/step - loss: 66263664.2760 - root_mean_squared_error: 7683.3737 - val_loss: 18541576.0000 - val_root_mean_squared_error: 4305.9917
Epoch 166/300
95/95 [==============================] - 0s 890us/step - loss: 25410102.6042 - root_mean_squared_error: 4672.7015 - val_loss: 18396066.0000 - val_root_mean_squared_error: 4289.0625
Epoch 167/300
95/95 [==============================] - 0s 859us/step - loss: 94881119.9167 - root_mean_squared_error: 9018.5368 - val_loss: 11989627.0000 - val_root_mean_squared_error: 3462.6023
Epoch 168/300
95/95 [=====

Epoch 205/300
95/95 [==============================] - 0s 897us/step - loss: 46792910.8646 - root_mean_squared_error: 6177.9108 - val_loss: 7731140.5000 - val_root_mean_squared_error: 2780.4910
Epoch 206/300
95/95 [==============================] - 0s 892us/step - loss: 90153388.1979 - root_mean_squared_error: 9194.6951 - val_loss: 5882699.5000 - val_root_mean_squared_error: 2425.4255
Epoch 207/300
95/95 [==============================] - 0s 886us/step - loss: 75237990.6667 - root_mean_squared_error: 8277.6704 - val_loss: 7280482.5000 - val_root_mean_squared_error: 2698.2349
Epoch 208/300
95/95 [==============================] - 0s 890us/step - loss: 126551891.2500 - root_mean_squared_error: 11029.3812 - val_loss: 6062514.0000 - val_root_mean_squared_error: 2462.2148
Epoch 209/300
95/95 [==============================] - 0s 854us/step - loss: 30078634.7500 - root_mean_squared_error: 5141.7109 - val_loss: 6913817.5000 - val_root_mean_squared_error: 2629.4119
Epoch 210/300
95/95 [=======

95/95 [==============================] - 0s 876us/step - loss: 59964755.7031 - root_mean_squared_error: 7221.5542 - val_loss: 5503039.5000 - val_root_mean_squared_error: 2345.8523
Epoch 290/300
95/95 [==============================] - 0s 888us/step - loss: 145574404.4583 - root_mean_squared_error: 11704.7827 - val_loss: 4550122.5000 - val_root_mean_squared_error: 2133.0981
Epoch 291/300
95/95 [==============================] - 0s 864us/step - loss: 47942110.6146 - root_mean_squared_error: 6494.6439 - val_loss: 5207830.0000 - val_root_mean_squared_error: 2282.0637
Epoch 292/300
95/95 [==============================] - 0s 873us/step - loss: 80581015.1823 - root_mean_squared_error: 8420.2536 - val_loss: 5242951.5000 - val_root_mean_squared_error: 2289.7458
Epoch 293/300
95/95 [==============================] - 0s 874us/step - loss: 35820880.1979 - root_mean_squared_error: 5794.2206 - val_loss: 5745257.0000 - val_root_mean_squared_error: 2396.9233
Epoch 294/300
95/95 [=====================

In [7]:
print('Root Mean Squared Error:', mse_test)

Root Mean Squared Error: 1703.6371
